In [22]:
# Install and update spaCy 
!pip install -U spacy

# Download the english language model
!python -m spacy download en

#Result to excel
!pip install openpyxl


You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
2023-04-18 10:50:37.933426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 10:50:38.055864: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 10:50:38.060628: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-18 10:

In [23]:
import os
import csv
import pandas as pd
import numpy as np
import math
from openpyxl import Workbook


from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline

In [3]:
version = 11

#with important_words_v1 w:0.1

#Settings: remove # to select, add # to unselect

## Select classifier

###Logistic Regression
#classifier=LogisticRegression()
###Random Forest Classifier
#classifier = RandomForestClassifier()
###Linear Support Vector Classification
classifier = LinearSVC()
#

## Ignore tokens? 

###No
IgnoreLimit = 0
###Yes, ignore token with coef under 0.05
#IgnoreLimit = 0.005
#

## Ignore most frequent tokens?

###No
#max_df=1 
###Ignore 0.5% most frequent tokens
#max_df=0.995
#



In [4]:
#Load the df generated in PrepareData
df = pd.read_csv('/datasets/cgmech/CGMech/df.csv')
del df[df.columns[0]]

df

,Filename,CGMech,content
0,ABB Group Annual Report 2015_English-2.txt,8,\n \n The ABB Group Annual Report 2015 \n \n \...
1,ABB_02.txt,8,ABB Group Annual Report 2002\n \n Financial re...
2,ABB_03.txt,8,Important information regarding the ABB 2003 A...
3,ABB_04.txt,8,ABB Annual Report 2004 \n \n Financial review ...
4,ABB_05.pdf.txt,9,i\n \n D\n e\n s\n g\n n\n e\n d\n \n \n \n b...
...,...,...,...
1694,WISeKey_2017.pdf.txt,9,ANNUAL REPORT 2017\n \n O I N \n \n - DIGI...
1695,WiSeKey-Annual-Report-2018.pdf.txt,9,Annual Report 2018\n \n ARCHITECTING A WISER W...
1696,ZurRoseGroup-2017-EN-Vollbericht.pdf.txt,9,Annual Report 2017\n \n Zur Rose Group \n \n ...
1697,ZurRoseGroup-2018-EN-Annual-Report.pdf.txt,9,Annual Report 2018\n \n Zur Rose Group \n \n ...


In [5]:
#Group CGMech by categories: 'low', 'middle', 'high'
def categorize(value):
    value=int(value)
    if value >= 1 and value <= 5:
        return 'low'
    elif value >= 6 and value <= 8:
        return 'middle'
    elif value >= 9 and value <= 10:
        return 'high'
    else:
        return 'unknown'

# apply function to create new "cat" column
df['cat'] = df['CGMech'].apply(categorize)

#split dataset: 80%train, 20% test
X = df.content
ylabels=df.CGMech
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=1232)


In [6]:
#visualize categories
cat_counts = df['cat'].value_counts()

print(cat_counts)
cat_percentages = cat_counts / len(df) * 100
print('Total: ', len(df))
print()
cat_percentages=cat_percentages.round(0)

print("Percentage:")
print(cat_percentages)

#Baserate
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(None, y_train)
baserate = dummy.score(None, y_test)
print("Baserate: ")
print(baserate)

high      912
middle    645
low       142
Name: cat, dtype: int64
Total:  1699

Percentage:
high      54.0
middle    38.0
low        8.0
Name: cat, dtype: float64
Baserate: 
0.31176470588235294


In [7]:
X

0       \n \n The ABB Group Annual Report 2015 \n \n \...
1       ABB Group Annual Report 2002\n \n Financial re...
2       Important information regarding the ABB 2003 A...
3       ABB Annual Report 2004 \n \n Financial review ...
4       i\n \n D\n e\n s\n g\n n\n e\n d\n \n  \n \n b...
                              ...                        
1694    ANNUAL REPORT 2017\n \n O I N  \n \n   -  DIGI...
1695    Annual Report 2018\n \n ARCHITECTING A WISER W...
1696    Annual Report 2017\n \n Zur Rose Group \n \n 
...
1697    Annual Report 2018\n \n Zur Rose Group \n \n 
...
1698    Annual Report 2019\n \n Zur Rose Group \n \n 
...
Name: content, Length: 1699, dtype: object

In [8]:
import spacy

#tokenize string (content of .txt doc) to word tokens
def spacyTokens(SpacyString):
    sp = spacy.load('en_core_web_sm')  #load English language model
    sp.max_length=6000000 #increase max length of document (len(logitech09.pdf.txt) is over 5M)
    doc = sp(SpacyString)
    cleaned_tokens = []
    for token in doc:
        if not token.is_stop and token.is_alpha and not token.is_digit and not token.is_punct:
            cleaned_token = token.lemma_.lower().strip()
            if cleaned_token:
                cleaned_tokens.append(cleaned_token)
    return cleaned_tokens



/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-04-18 06:55:33.447916: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-18 06:55:33.563706: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-18 06:55:33.568256: W tensorflow/stream_executor/platform/default/dso_loader.

In [9]:
# Step 1: Load the CSV file
important_words = {}
with open('important_words.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header
    for row in reader:
        word = row[0]
        weight = 0.1 #float(row[1])
        important_words[word]= weight

print(important_words)        

# Custom IDF function to adjust word weights
def custom_idf(doc_freq, total_docs):
    adjusted_idf = {}
    for word, freq in doc_freq.items():
        if word in important_words:
            adjusted_idf[word] = np.log(float(total_docs) / freq) * important_words[word]
        else:
            adjusted_idf[word] = np.log(float(total_docs) / freq)
    return adjusted_idf

class CustomTfidfVectorizer(TfidfVectorizer):
    def __init__(self, tokenizer):
        super().__init__(tokenizer=tokenizer)

    def fit(self, raw_documents, y=None):
        super().fit(raw_documents, y)
        idf = custom_idf(self.vocabulary_, len(raw_documents))
        self._tfidf._idf_diag = sp.spdiags(np.array(list(idf.values())), diags=0, m=len(idf), n=len(idf))
        return self


pipe = Pipeline([
    ('vectorizer', CustomTfidfVectorizer(tokenizer=spacyTokens)),
    ('classifier', LinearSVC())])


{'efficiency': 0.1, 'specific': 0.1, 'engage': 0.1, 'participate': 0.1, 'culture': 0.1, 'role': 0.1, 'driver': 0.1, 'connect': 0.1, 'knowledge': 0.1, 'protect': 0.1, 'equal': 0.1, 'reflect': 0.1, 'code': 0.1, 'action': 0.1, 'regular': 0.1, 'regulatory': 0.1, 'understanding': 0.1, 'skill': 0.1, 'evaluate': 0.1, 'innovate': 0.1, 'accountable': 0.1, 'stakeholder': 0.1, 'professional background': 0.1, 'nomination': 0.1, 'succession': 0.1, 'multiple': 0.1, 'engagement': 0.1, 'tenure': 0.1}


In [10]:
print("fitting...")
pipe.fit(X_train, y_train)

fitting...


Pipeline(steps=[('vectorizer',
                 CustomTfidfVectorizer(tokenizer=<function spacyTokens at 0x7f2aea24eb80>)),
                ('classifier', LinearSVC())])

In [11]:
vectorizer = pipe.named_steps['vectorizer']
classifier = pipe.named_steps['classifier']
feature_names = vectorizer.get_feature_names()

#indices_to_keep = pipe.named_steps['remove_frequent'].indices_to_keep
#feature_names = np.array(vectorizer.get_feature_names())[indices_to_keep]

coefs = classifier.coef_[0].tolist()



/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
len(feature_names)

131452

In [13]:
len(coefs)

131452

In [14]:
# Extract tokens
all_tokens = [(feature_names[i], round(coefs[i], 2)) for i in range(len(feature_names))]
#all_tokens = [(feature_names[i], round(coefs[i], 2)) for i in range(len(feature_names)) if str(round(coefs[i],2)) != '0.0' and str(round(coefs[i],2)) != '-0.0']
print(all_tokens)

# Write all tokens to CSV
with open('all_tokens_'+str(version)+'.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for token, coef in all_tokens:
        writer.writerow([token, coef])

tag', 0.0), ('satara', 0.0), ('satay', 0.0), ('satchel', 0.0), ('sate', 0.0), ('satel', 0.0), ('satellite', -0.0), ('satellitentechnik', 0.0), ('satellitesdure', 0.0), ('satellitesmaintain', 0.0), ('satellitesor', 0.0), ('satement', 0.0), ('sateur', 0.0), ('sathanandan', 0.0), ('sathd', 0.0), ('sathorn', 0.0), ('sathyaseelan', 0.0), ('satiation', 0.0), ('satigny', 0.0), ('satile', 0.0), ('satility', 0.0), ('satimat', 0.0), ('satin', -0.0), ('satinage', 0.0), ('satine', 0.0), ('sating', 0.0), ('satintour', 0.0), ('satinvest', 0.0), ('sation', 0.0), ('sational', -0.0), ('sationally', -0.0), ('sationen', 0.0), ('satire', 0.0), ('satirical', 0.0), ('satis', 0.0), ('satisfac', 0.0), ('satisfaction', 0.01), ('satisfactionsurvey', 0.0), ('satisfactorily', -0.0), ('satisfactory', 0.0), ('satisfaire', 0.0), ('satisfaisante', -0.0), ('satisfait', 0.0), ('satisfaite', 0.0), ('satisfaites', 0.0), ('satisfecit', 0.0), ('satisfi', 0.0), ('satisfie', -0.0), ('satisfied', -0.0), ('satisfont', 0.0), ('

In [15]:
#Positive
input_file = 'all_tokens_'+str(version)+'.csv'
output_file = 'top_positive_tokens_'+str(version)+'.csv'

# Read input CSV file and filter rows
filtered_rows = []
with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header row
    for row in reader:
        if float(row[1]) >= IgnoreLimit:
            filtered_rows.append(row)
print(filtered_rows)

# Write filtered rows to output CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for row in filtered_rows:
        writer.writerow(row)


.0'], ['serte', '0.0'], ['sertec', '0.0'], ['sertig', '0.0'], ['sertion', '0.0'], ['sertissage', '-0.0'], ['serto', '0.01'], ['sertürner', '0.0'], ['serum', '-0.0'], ['serumwerk', '0.0'], ['serung', '0.0'], ['serv', '-0.0'], ['servable', '-0.0'], ['serval', '0.0'], ['servancy', '0.0'], ['servant', '-0.0'], ['servation', '-0.0'], ['servationist', '0.0'], ['servative', '0.0'], ['servatively', '0.0'], ['servea', '0.0'], ['servehedging', '0.0'], ['serven', '0.0'], ['server', '-0.0'], ['servet', '0.0'], ['servi', '-0.0'], ['servic', '0.0'], ['service', '0.01'], ['serviceability', '0.0'], ['serviceable', '0.0'], ['serviceby', '0.0'], ['servicecare', '0.0'], ['servicecenter', '0.01'], ['servicecharacterize', '0.0'], ['serviced', '0.0'], ['servicedesk', '0.0'], ['servicedienstleistungen', '0.0'], ['serviceengineer', '0.0'], ['servicefachleute', '0.0'], ['servicegeschäft', '0.0'], ['servicegesellschaft', '0.0'], ['serviceman', '0.0'], ['servicemax', '0.0'], ['servicemen', '-0.0'], ['servicenet'

In [16]:
#Negative
input_file = 'all_tokens_'+str(version)+'.csv'
output_file = 'top_negative_tokens_'+str(version)+'.csv'

# Read input CSV file and filter rows
filtered_rows = []
with open(input_file, 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # skip header row
    for row in reader:
        if float(row[1]) <= -IgnoreLimit:
            filtered_rows.append(row)
print(filtered_rows)


# Write filtered rows to output CSV file
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Token', 'Coefficient'])
    for row in filtered_rows:
        writer.writerow(row)


'sertürner', '0.0'], ['serum', '-0.0'], ['serumwerk', '0.0'], ['serung', '0.0'], ['serv', '-0.0'], ['servable', '-0.0'], ['serval', '0.0'], ['servancy', '0.0'], ['servant', '-0.0'], ['servation', '-0.0'], ['servationist', '0.0'], ['servative', '0.0'], ['servatively', '0.0'], ['serve', '-0.01'], ['servea', '0.0'], ['servehedging', '0.0'], ['serven', '0.0'], ['server', '-0.0'], ['servet', '0.0'], ['servi', '-0.0'], ['servic', '0.0'], ['serviceability', '0.0'], ['serviceable', '0.0'], ['serviceby', '0.0'], ['servicecare', '0.0'], ['servicecharacterize', '0.0'], ['serviced', '0.0'], ['servicedesk', '0.0'], ['servicedienstleistungen', '0.0'], ['serviceengineer', '0.0'], ['servicefachleute', '0.0'], ['servicegeschäft', '0.0'], ['servicegesellschaft', '0.0'], ['serviceman', '0.0'], ['servicemax', '0.0'], ['servicemen', '-0.0'], ['servicenet', '0.0'], ['serviceorganization', '0.0'], ['serviceplan', '0.0'], ['serviceport', '0.0'], ['serviceprodukte', '0.0'], ['servicepurpose', '0.0'], ['service

In [17]:
#Test on the remaining 20% of the dataset: 
#make the model predict and compare with real

print('making predictions..')
y_pred=pipe.predict(X_test)

making predictions..


In [24]:
def evaluate(true, pred):
    # Evaluate model
    accuracy = accuracy_score(true, pred)
    precision = precision_score(true, pred, average='weighted')
    recall = recall_score(true, pred, average='weighted')
    f1 = f1_score(true, pred, average='weighted')
    cm = confusion_matrix(true, pred)
 
    # Create Excel workbook
    wb = Workbook()
    ws1 = wb.active
    ws1.title = "EvalCommit"+str(version)
    
    # Write variables to sheet 1
    ws1["A1"] = "Accuracy"
    ws1["B1"] = "{:.2%}".format(accuracy)
    ws1["A2"] = "Precision"
    ws1["B2"] = "{:.2%}".format(precision)
    ws1["A3"] = "Recall"
    ws1["B3"] = "{:.2%}".format(recall)
    ws1["A4"] = "F1 Score"
    ws1["B4"] = "{:.2%}".format(f1)
    
    # Create sheet 2 and write confusion matrix
    ws2 = wb.create_sheet("Confusion Matrix"+str(version))
    for row in cm.tolist():
        ws2.append(row)

    # Add a sheet for top negative tokens
    negative_sheet = wb.create_sheet(title="Top Negative Tokens"+str(version))
    with open('top_negative_tokens_'+str(version)+'.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        sorted_rows = sorted(reader, key=lambda row: float(row[1]))
        negative_sheet['A1'] = "Token"
        negative_sheet['B1'] = "Coefficient"
        for idx, row in enumerate(sorted_rows):
            negative_sheet.cell(row=idx+2, column=1, value=row[0])
            negative_sheet.cell(row=idx+2, column=2, value=row[1])

    # Add a sheet for top positive tokens
    positive_sheet = wb.create_sheet(title="Top Positive Tokens"+str(version))
    with open('top_positive_tokens_'+str(version)+'.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # skip header row
        sorted_rows = sorted(reader, key=lambda row: float(row[1]), reverse=True)
        positive_sheet['A1'] = "Token"
        positive_sheet['B1'] = "Coefficient"
        for idx, row in enumerate(sorted_rows):
            positive_sheet.cell(row=idx+2, column=1, value=row[0])
            positive_sheet.cell(row=idx+2, column=2, value=row[1])
    
    # Save workbook
    wb.save("EvalCommit"+str(version)+".xlsx")
    
    # Print evaluation metrics
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{cm}")


evaluate(y_test, y_pred)


/shared-libs/python3.9/py/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Accuracy: 0.6118
Precision: 0.6018
Recall: 0.6118
F1 Score: 0.5985
Confusion Matrix:
[[ 0  1  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0]
 [ 0  0  4  2  1  0  1  0  0]
 [ 0  0  4  5  0  1  0  0  3]
 [ 0  0  0  3  7  8  1  0  6]
 [ 0  0  0  1  1 13 10  5  6]
 [ 0  0  0  0  2  4 36 15  8]
 [ 0  1  0  0  0  2  7 56 17]
 [ 0  0  0  0  0  0  4 15 87]]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=961bc5f7-68db-4917-95e0-61c59b88476b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>